### Snap Overlaps

In [1]:
import geopandas as gpd

In [6]:
buildings_trimmed = gpd.read_parquet('buildings_trimmed.parquet')

In [7]:
overlaps = buildings_trimmed.sindex.query(buildings_trimmed.geometry, predicate="overlaps")

In [9]:
buildings_trimmed.loc[overlaps[0]].explore(tiles=None)

In [10]:
import pandas as pd
import numpy as np
import geopandas
import shapely
from packaging.version import Version

__all__ = ["gaps", "fill_gaps", "snap"]

GPD_GE_014 = Version(geopandas.__version__) >= Version("0.14.0")
GPD_GE_100 = Version(geopandas.__version__) >= Version("1.0.0dev")

def _snap(geometry, reference, threshold, segment_length):
    """Snap g1 to g2 within threshold

    Parameters
    ----------
    geometry : shapely.Polygon
        geometry to snap
    reference : shapely.Polygon
        geometry to snap to
    threshold : float
        max distance between vertices to snap
    segment_length : float
        max segment length parameter in segmentize

    Returns
    -------
    shapely.Polygon
        snapped geometry
    """

    # extract the shell and holes from the first geometry
    shell, holes = geometry.exterior, geometry.interiors
    # segmentize the shell and extract coordinates
    coords = shapely.get_coordinates(shapely.segmentize(shell, segment_length))
    # create a point geometry from the coordinates
    points = shapely.points(coords)
    # find the shortest line between the points and the second geometry
    lines = shapely.shortest_line(points, reference)
    # mask the coordinates where the distance is less than the threshold
    distance_mask = shapely.length(lines) < threshold

    # return the original geometry if no coordinates are within the threshold
    if not any(distance_mask):
        return geometry

    # update the coordinates with the snapped coordinates
    coords[distance_mask] = shapely.get_coordinates(lines)[1::2][distance_mask]
    # re-create the polygon with new coordinates and original holes and simplify
    # to remove any extra vertices
    return shapely.simplify(shapely.Polygon(coords, holes=holes), segment_length / 100)


def snap_overlaps(geometry, threshold):
    """Snap overlapping geometries

    Only one of the pair of geometries identified as nearby will be snapped,
    the one with the lower index.

    Parameters
    ----------
    geometry : GeoDataFrame | GeoSeries
        geometries to snap. Geometry type needs to be Polygon for all of them.
    threshold : float
        max distance between geometries to snap
        threshold should be ~10% larger than the distance between polygon edges to
        ensure snapping

    Returns
    -------
    GeoSeries
        GeoSeries with snapped geometries
    """

    overlapping_a, overlapping_b = geometry.sindex.query(geometry.geometry, predicate="overlaps")

    self_mask = overlapping_a != overlapping_b
    overlapping_a = overlapping_a[self_mask]
    overlapping_b = overlapping_b[self_mask]

    overlapping = pd.MultiIndex.from_arrays([overlapping_a, overlapping_b], names=("source", "target"))

    if not overlapping.empty:
        duplicated = pd.DataFrame(
            np.sort(np.array(overlapping.to_list()), axis=1)
        ).duplicated()
        pairs_to_snap = overlapping[~duplicated]

        new_geoms = []
        previous_geom = None
        snapped_geom = None
        for geom, ref in zip(
            geometry.geometry.iloc[pairs_to_snap.get_level_values("source")],
            geometry.geometry.iloc[pairs_to_snap.get_level_values("target")],
            strict=True,
        ):
            if previous_geom == geom:
                new_geoms.append(
                    _snap(
                        snapped_geom, ref, threshold=threshold, segment_length=threshold
                    )
                )
            else:
                snapped_geom = _snap(
                    geom, ref, threshold=threshold, segment_length=threshold
                )
                new_geoms.append(snapped_geom)
                previous_geom = geom

        snapped = geometry.geometry.copy()
        snapped.iloc[pairs_to_snap.get_level_values("source")] = new_geoms
    else:
        snapped = geometry.geometry.copy()
    return snapped

In [11]:
buildings_trimmed_snapped=snap_overlaps(buildings_trimmed, 0.5)

In [12]:
overlaps_snapped = buildings_trimmed_snapped.sindex.query(buildings_trimmed_snapped.geometry, predicate="overlaps")

In [13]:
overlaps_snapped

array([[  10703,   15527,   15528,   43723,   49302,   78163,  113432,
         152336,  152336,  181402,  181402,  213550,  213551,  577171,
         603176,  603177,  726083,  816732,  864622,  864626, 1051914,
        1053996, 1054601, 1055057, 1055059, 1055363, 1056372, 1056373],
       [1051914,   15528,   15527,   49302,   43723,  726083,  816732,
        1056372, 1056373,  213550,  213551,  181402,  181402, 1055363,
         603177,  603176,   78163,  113432,  864626,  864622,   10703,
        1054601, 1053996, 1055059, 1055057,  577171,  152336,  152336]])